# Federated Views: Agent-Ready Neo4j + Delta Lakehouse

Creates Unity Catalog views that encapsulate `remote_query()` calls to Neo4j, making
graph data queryable as regular UC tables. These views enable **Genie** (or any SQL tool)
to transparently federate queries across Neo4j and Delta lakehouse without Spark Connector
or direct Python drivers.

The full chain: **Natural Language → SQL (Genie) → Cypher (JDBC driver) → Neo4j** — all
through Unity Catalog federation.

## Architecture

```
┌──────────────────────────────────────────────────────────────────────┐
│                  Genie / SQL Tool / Agent                           │
│           (natural language or SQL queries)                         │
└─────────────────────────────┬────────────────────────────────────────┘
                              │
                              ▼
┌──────────────────────────────────────────────────────────────────────┐
│                      Spark SQL Engine                               │
│                                                                      │
│   Queries run against UC tables (all look like regular tables):     │
│   - aircraft, systems, sensors, sensor_readings  (Delta — direct)   │
│   - neo4j_maintenance_events  (materialized from Neo4j via JDBC)    │
│   - neo4j_flights             (materialized from Neo4j via JDBC)    │
│   - neo4j_airports            (materialized from Neo4j via JDBC)    │
│   - neo4j_flight_airports     (JOIN of flights + airports)          │
│                                                                      │
├────────────────────────────┬─────────────────────────────────────────┤
│  Delta Lakehouse (direct)  │  Neo4j (materialized via JDBC dbtable) │
│                            │  Read via dbtable + customSchema       │
│                            │  Written as managed Delta tables       │
│                            │  Re-run notebook to refresh            │
└────────────────────────────┴─────────────────────────────────────────┘
```

**Key advantage:** GROUP BY, ORDER BY, and JOINs all work because the Neo4j data is
materialized as regular Delta tables. Genie sees all 8 tables identically.

**Why materialized?** Neo4j JDBC has two schema inference issues that prevent live
`CREATE VIEW` over `remote_query()`:
1. `query` option: Spark wraps in subquery → Neo4j can't parse it
2. `dbtable` option: Neo4j returns NullType → requires `customSchema` (DataFrame API only)

## Prerequisites

1. **Lakehouse tables** already exist in your catalog (created by lab setup):
   `aircraft`, `systems`, `sensors`, `sensor_readings`

2. **Neo4j UC JDBC connection** configured per [neo4j_uc_jdbc_guide.md](../docs/neo4j_uc_jdbc_guide.md)

3. **Cluster configuration:**
   - SafeSpark memory settings applied (see guide)
   - `neo4j-uc-creds` secret scope configured via `setup.sh`

4. **Databricks preview features** enabled:
   - Custom JDBC on UC Compute
   - `remote_query` table-valued function

**Note:** No Spark Connector or cluster libraries required — this notebook uses
pure UC JDBC federation only.

---

## Configuration

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

SCOPE_NAME = "neo4j-uc-creds"

# Lakehouse configuration — update to match your environment
LAKEHOUSE_CATALOG = "aws-databricks-neo4j-lab"   # Your Unity Catalog name
LAKEHOUSE_SCHEMA = "lakehouse"                    # Schema containing Delta tables

# Neo4j credentials from Databricks Secrets
NEO4J_HOST = dbutils.secrets.get(SCOPE_NAME, "host")
NEO4J_USER = dbutils.secrets.get(SCOPE_NAME, "user")
NEO4J_PASSWORD = dbutils.secrets.get(SCOPE_NAME, "password")
try:
    NEO4J_DATABASE = dbutils.secrets.get(SCOPE_NAME, "database")
except Exception:
    NEO4J_DATABASE = "neo4j"

UC_CONNECTION_NAME = dbutils.secrets.get(SCOPE_NAME, "connection_name")

# Set catalog and schema context
spark.sql(f"USE CATALOG `{LAKEHOUSE_CATALOG}`")
spark.sql(f"USE SCHEMA `{LAKEHOUSE_SCHEMA}`")

print(f"Lakehouse: {LAKEHOUSE_CATALOG}.{LAKEHOUSE_SCHEMA}")
print(f"Neo4j Host: {NEO4J_HOST}")
print(f"UC Connection: {UC_CONNECTION_NAME}")

---

## Section 1: Verify Data Sources

Confirm both the lakehouse tables and Neo4j UC connection are accessible.

In [ ]:
# Verify Delta lakehouse tables
print("=" * 60)
print("DELTA LAKEHOUSE TABLES")
print("=" * 60)

for table in ["aircraft", "systems", "sensors", "sensor_readings"]:
    count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {table}").collect()[0]["cnt"]
    print(f"  {table}: {count:,} rows")

print("\nSample aircraft data:")
spark.sql("""
    SELECT `:ID(Aircraft)` AS aircraft_id, tail_number, model, manufacturer, operator
    FROM aircraft LIMIT 5
""").show(truncate=False)

In [ ]:
# Verify Neo4j UC JDBC connection
print("=" * 60)
print("NEO4J KNOWLEDGE GRAPH (via UC JDBC)")
print("=" * 60)

neo4j_counts = {
    "Aircraft": "SELECT COUNT(*) AS cnt FROM Aircraft",
    "MaintenanceEvent": "SELECT COUNT(*) AS cnt FROM MaintenanceEvent",
    "Flight": "SELECT COUNT(*) AS cnt FROM Flight",
    "Airport": "SELECT COUNT(*) AS cnt FROM Airport",
}

for label, query in neo4j_counts.items():
    result = spark.sql(f"""
        SELECT * FROM remote_query('{UC_CONNECTION_NAME}', query => '{query}')
    """).collect()
    print(f"  {label}: {result[0]['cnt']:,} nodes")

print("\nBoth data sources verified.")

---

## Section 2: Materialize Neo4j Data as UC Delta Tables

Each Neo4j label is read via the DataFrame API with `dbtable` + `customSchema`, then
written as a **managed Delta table** in Unity Catalog. These tables are visible in
Catalog Explorer and can be added to a Genie space for natural language querying.

**Why `dbtable` + `customSchema`?** Two Neo4j JDBC limitations require this approach:
1. The `query` option triggers Spark's subquery wrapping for schema inference, which
   Neo4j's SQL translator cannot handle.
2. The `dbtable` option avoids subquery wrapping but Neo4j JDBC returns `NullType`
   during schema inference, requiring `customSchema` to specify column types explicitly.

**Why materialized tables?** The `customSchema` fix is only available on the DataFrame
API (`spark.read.format("jdbc")`), not on `remote_query()`. Since `CREATE VIEW` requires
`remote_query()`, we materialize the data as Delta tables instead. Re-run this section
to refresh data from Neo4j.

In [ ]:
# =============================================================================
# TABLE: neo4j_maintenance_events (materialized Delta table)
# =============================================================================
# Reads MaintenanceEvent nodes from Neo4j via dbtable + customSchema, then
# writes as a managed Delta table in Unity Catalog.
# customSchema is required because Neo4j JDBC returns NullType during inference.
# Re-run this cell to refresh data from Neo4j.

TABLE_NAME = f"`{LAKEHOUSE_CATALOG}`.`{LAKEHOUSE_SCHEMA}`.neo4j_maintenance_events"

# Drop any existing view or table to allow overwrite
spark.sql(f"DROP VIEW IF EXISTS {TABLE_NAME}")
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

MAINTENANCE_SCHEMA = """`v$id` STRING, aircraft_id STRING, system_id STRING, component_id STRING,
    event_id STRING, severity STRING, fault STRING, corrective_action STRING, reported_at STRING"""

df = spark.read.format("jdbc") \
    .option("databricks.connection", UC_CONNECTION_NAME) \
    .option("dbtable", "MaintenanceEvent") \
    .option("customSchema", MAINTENANCE_SCHEMA) \
    .load() \
    .select("aircraft_id", "fault", "severity", "corrective_action", "reported_at")

df.write.mode("overwrite").saveAsTable(TABLE_NAME)

count = spark.sql("SELECT COUNT(*) AS cnt FROM neo4j_maintenance_events").collect()[0]["cnt"]
print(f"neo4j_maintenance_events: {count:,} rows")

print("\nSchema:")
spark.sql("SELECT * FROM neo4j_maintenance_events LIMIT 1").printSchema()

print("Sample data:")
spark.sql("""
    SELECT aircraft_id, fault, severity, corrective_action
    FROM neo4j_maintenance_events LIMIT 5
""").show(truncate=False)

In [ ]:
# =============================================================================
# TABLE: neo4j_flights (materialized Delta table)
# =============================================================================
# Reads Flight nodes from Neo4j via dbtable + customSchema, then writes as a
# managed Delta table. Re-run this cell to refresh data from Neo4j.

TABLE_NAME = f"`{LAKEHOUSE_CATALOG}`.`{LAKEHOUSE_SCHEMA}`.neo4j_flights"

# Drop any existing view or table to allow overwrite
spark.sql(f"DROP VIEW IF EXISTS {TABLE_NAME}")
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

FLIGHT_SCHEMA = """`v$id` STRING, aircraft_id STRING, flight_id STRING, operator STRING,
    flight_number STRING, origin STRING, destination STRING,
    scheduled_departure STRING, scheduled_arrival STRING"""

df = spark.read.format("jdbc") \
    .option("databricks.connection", UC_CONNECTION_NAME) \
    .option("dbtable", "Flight") \
    .option("customSchema", FLIGHT_SCHEMA) \
    .load() \
    .select("aircraft_id", "flight_number", "operator", "origin", "destination",
            "scheduled_departure", "scheduled_arrival")

df.write.mode("overwrite").saveAsTable(TABLE_NAME)

count = spark.sql("SELECT COUNT(*) AS cnt FROM neo4j_flights").collect()[0]["cnt"]
print(f"neo4j_flights: {count:,} rows")

print("\nSchema:")
spark.sql("SELECT * FROM neo4j_flights LIMIT 1").printSchema()

print("Sample data:")
spark.sql("""
    SELECT aircraft_id, flight_number, operator, origin, destination
    FROM neo4j_flights LIMIT 5
""").show(truncate=False)

In [ ]:
# =============================================================================
# TABLES: neo4j_airports + neo4j_flight_airports (materialized Delta tables)
# =============================================================================
# Reads Airport nodes from Neo4j via dbtable + customSchema, then creates a
# flight-to-airport mapping table via Spark SQL JOIN.
# Re-run this cell to refresh data from Neo4j.

from pyspark.sql.functions import col

AIRPORTS_TABLE = f"`{LAKEHOUSE_CATALOG}`.`{LAKEHOUSE_SCHEMA}`.neo4j_airports"
FLIGHT_AIRPORTS_TABLE = f"`{LAKEHOUSE_CATALOG}`.`{LAKEHOUSE_SCHEMA}`.neo4j_flight_airports"

# Drop any existing views or tables to allow overwrite
spark.sql(f"DROP VIEW IF EXISTS {FLIGHT_AIRPORTS_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {FLIGHT_AIRPORTS_TABLE}")
spark.sql(f"DROP VIEW IF EXISTS {AIRPORTS_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {AIRPORTS_TABLE}")

AIRPORT_SCHEMA = """`v$id` STRING, airport_id STRING, name STRING, city STRING,
    country STRING, iata STRING, icao STRING, lat STRING, lon STRING"""

airports_df = spark.read.format("jdbc") \
    .option("databricks.connection", UC_CONNECTION_NAME) \
    .option("dbtable", "Airport") \
    .option("customSchema", AIRPORT_SCHEMA) \
    .load() \
    .select("iata", col("name").alias("airport_name"), "city", "country", "icao", "lat", "lon")

airports_df.write.mode("overwrite").saveAsTable(AIRPORTS_TABLE)

# Create flight_airports as a JOIN of the two materialized tables
spark.sql(f"""
    CREATE OR REPLACE TABLE {FLIGHT_AIRPORTS_TABLE} AS
    SELECT f.flight_number, f.aircraft_id, a.iata AS airport_code, a.airport_name
    FROM `{LAKEHOUSE_CATALOG}`.`{LAKEHOUSE_SCHEMA}`.neo4j_flights f
    JOIN `{LAKEHOUSE_CATALOG}`.`{LAKEHOUSE_SCHEMA}`.neo4j_airports a ON f.origin = a.iata
""")

count = spark.sql("SELECT COUNT(*) AS cnt FROM neo4j_flight_airports").collect()[0]["cnt"]
print(f"neo4j_flight_airports: {count:,} rows")

print("\nSchema:")
spark.sql("SELECT * FROM neo4j_flight_airports LIMIT 1").printSchema()

print("Sample data:")
spark.sql("""
    SELECT flight_number, aircraft_id, airport_code, airport_name
    FROM neo4j_flight_airports LIMIT 5
""").show(truncate=False)

In [ ]:
# Verify all materialized Neo4j tables are accessible
print("=" * 60)
print("ALL NEO4J TABLES MATERIALIZED")
print("=" * 60)
print(f"Catalog: {LAKEHOUSE_CATALOG}.{LAKEHOUSE_SCHEMA}\n")

neo4j_tables = {
    "neo4j_maintenance_events": "Neo4j MaintenanceEvent nodes",
    "neo4j_flights": "Neo4j Flight nodes",
    "neo4j_airports": "Neo4j Airport nodes",
    "neo4j_flight_airports": "Spark SQL JOIN of flights + airports",
}

for table_name, description in neo4j_tables.items():
    count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {table_name}").collect()[0]["cnt"]
    print(f"  {table_name}: {count:,} rows  ({description})")

print("\nDelta tables (direct from lakehouse):")
for table in ["aircraft", "systems", "sensors", "sensor_readings"]:
    count = spark.sql(f"SELECT COUNT(*) AS cnt FROM {table}").collect()[0]["cnt"]
    print(f"  {table}: {count:,} rows")

print(f"\nAll 8 tables are ready for federated queries and Genie.")

---

## Section 3: SQL Tests Against Views

Test that the Neo4j views support standard SQL operations — GROUP BY, ORDER BY,
WHERE filters, aggregations — that would fail with inline `remote_query()` calls.
These are the operations Genie needs to generate.

In [ ]:
# =============================================================================
# TEST 1: GROUP BY on Neo4j view
# =============================================================================
# This fails with inline remote_query() but works with views.

print("TEST 1: GROUP BY — Maintenance events by severity")
print("=" * 60)

result = spark.sql("""
    SELECT
        severity,
        COUNT(*) AS event_count
    FROM neo4j_maintenance_events
    GROUP BY severity
    ORDER BY event_count DESC
""")

result.show(truncate=False)
assert result.count() > 0, "Expected at least one severity level"
print("[PASS] GROUP BY on Neo4j view works")

In [ ]:
# =============================================================================
# TEST 2: ORDER BY on Neo4j view
# =============================================================================

print("TEST 2: ORDER BY — Flights ordered by flight_number")
print("=" * 60)

result = spark.sql("""
    SELECT flight_number, aircraft_id, operator, origin, destination
    FROM neo4j_flights
    ORDER BY flight_number
    LIMIT 10
""")

result.show(truncate=False)
rows = result.collect()
flight_numbers = [r["flight_number"] for r in rows]
assert flight_numbers == sorted(flight_numbers), "Expected sorted flight numbers"
print("[PASS] ORDER BY on Neo4j view works")

In [ ]:
# =============================================================================
# TEST 3: WHERE filter on Neo4j view
# =============================================================================

print("TEST 3: WHERE — Critical maintenance events only")
print("=" * 60)

result = spark.sql("""
    SELECT aircraft_id, fault, severity, corrective_action
    FROM neo4j_maintenance_events
    WHERE severity = 'CRITICAL'
""")

result.show(truncate=False)
count = result.count()
print(f"Found {count} CRITICAL events")

# Verify all returned rows are CRITICAL
severities = [r["severity"] for r in result.collect()]
assert all(s == "CRITICAL" for s in severities), "Expected all CRITICAL"
print("[PASS] WHERE filter on Neo4j view works")

In [ ]:
# =============================================================================
# TEST 4: Aggregations on Neo4j view
# =============================================================================

print("TEST 4: Aggregations — Maintenance stats per aircraft")
print("=" * 60)

result = spark.sql("""
    SELECT
        aircraft_id,
        COUNT(*) AS total_events,
        SUM(CASE WHEN severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical,
        SUM(CASE WHEN severity = 'MAJOR' THEN 1 ELSE 0 END) AS major,
        SUM(CASE WHEN severity = 'MINOR' THEN 1 ELSE 0 END) AS minor
    FROM neo4j_maintenance_events
    GROUP BY aircraft_id
    ORDER BY total_events DESC
    LIMIT 10
""")

result.show(truncate=False)
assert result.count() > 0, "Expected maintenance stats"
print("[PASS] Aggregations on Neo4j view work")

In [ ]:
# =============================================================================
# TEST 5: DISTINCT on Neo4j view
# =============================================================================

print("TEST 5: DISTINCT — Unique airports from graph traversal")
print("=" * 60)

result = spark.sql("""
    SELECT DISTINCT airport_code, airport_name
    FROM neo4j_flight_airports
    ORDER BY airport_code
""")

result.show(truncate=False)
count = result.count()
print(f"Found {count} unique airports")
assert count > 0, "Expected at least one airport"
print("[PASS] DISTINCT on Neo4j graph traversal view works")

In [ ]:
# =============================================================================
# TEST 6: GROUP BY on graph traversal view
# =============================================================================

print("TEST 6: GROUP BY — Flights per departure airport")
print("=" * 60)

result = spark.sql("""
    SELECT
        airport_code,
        airport_name,
        COUNT(*) AS departure_count
    FROM neo4j_flight_airports
    GROUP BY airport_code, airport_name
    ORDER BY departure_count DESC
""")

result.show(truncate=False)
assert result.count() > 0, "Expected at least one airport with departures"
print("[PASS] GROUP BY on graph traversal view works")

---

## Section 4: Federated Queries — Views + Delta Tables

The real power: JOIN Neo4j views with Delta lakehouse tables in a single query.
These are the same federated patterns from `federated_lakehouse_query.ipynb`, but
now using views instead of Spark Connector — purely through UC federation.

In [ ]:
# =============================================================================
# FEDERATED QUERY 1: Fleet Summary
# =============================================================================
# Combines Neo4j maintenance/flight counts with Delta sensor averages.
# Equivalent to Section 2 of federated_lakehouse_query.ipynb but using views.

print("Federated Query 1: Fleet Summary")
print("Neo4j views + Delta sensor_readings")
print("=" * 80)

result = spark.sql("""
    SELECT
        maint.total_maintenance_events,
        maint.critical_events,
        flights.total_flights,
        airports.unique_airports,
        ROUND(sensor.avg_egt, 1) AS avg_egt_celsius,
        ROUND(sensor.avg_vibration, 4) AS avg_vibration_ips,
        ROUND(sensor.avg_fuel_flow, 2) AS avg_fuel_flow_kgs,
        ROUND(sensor.avg_n1_speed, 0) AS avg_n1_speed_rpm,
        sensor.total_readings
    FROM (
        SELECT
            COUNT(*) AS total_maintenance_events,
            SUM(CASE WHEN severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical_events
        FROM neo4j_maintenance_events
    ) maint
    CROSS JOIN (
        SELECT COUNT(*) AS total_flights
        FROM neo4j_flights
    ) flights
    CROSS JOIN (
        SELECT COUNT(DISTINCT airport_code) AS unique_airports
        FROM neo4j_flight_airports
    ) airports
    CROSS JOIN (
        SELECT
            AVG(CASE WHEN sen.type = 'EGT' THEN r.value END) AS avg_egt,
            AVG(CASE WHEN sen.type = 'Vibration' THEN r.value END) AS avg_vibration,
            AVG(CASE WHEN sen.type = 'FuelFlow' THEN r.value END) AS avg_fuel_flow,
            AVG(CASE WHEN sen.type = 'N1Speed' THEN r.value END) AS avg_n1_speed,
            COUNT(*) AS total_readings
        FROM sensor_readings r
        JOIN sensors sen ON r.sensor_id = sen.`:ID(Sensor)`
    ) sensor
""")

result.show(truncate=False)
print("[PASS] Fleet summary federated query works")

In [ ]:
# =============================================================================
# FEDERATED QUERY 2: Sensor Health + Maintenance Correlation
# =============================================================================
# Per-aircraft correlation of sensor health (Delta) with maintenance events (Neo4j).
# Equivalent to Section 3 of federated_lakehouse_query.ipynb.

print("Federated Query 2: Sensor Health + Maintenance Correlation")
print("Delta: sensor_readings, sensors, systems, aircraft")
print("Neo4j: neo4j_maintenance_events (view)")
print("=" * 100)

result = spark.sql("""
    WITH aircraft_ref AS (
        SELECT `:ID(Aircraft)` AS aircraft_id, tail_number, model, manufacturer, operator
        FROM aircraft
    ),
    sensor_health AS (
        SELECT
            sys.aircraft_id,
            ROUND(AVG(CASE WHEN sen.type = 'EGT' THEN r.value END), 1) AS avg_egt,
            ROUND(MAX(CASE WHEN sen.type = 'EGT' THEN r.value END), 1) AS max_egt,
            ROUND(AVG(CASE WHEN sen.type = 'Vibration' THEN r.value END), 4) AS avg_vibration,
            ROUND(MAX(CASE WHEN sen.type = 'Vibration' THEN r.value END), 4) AS max_vibration
        FROM sensor_readings r
        JOIN sensors sen ON r.sensor_id = sen.`:ID(Sensor)`
        JOIN systems sys ON sen.system_id = sys.`:ID(System)`
        GROUP BY sys.aircraft_id
    ),
    maintenance_summary AS (
        SELECT
            aircraft_id,
            COUNT(*) AS total_events,
            SUM(CASE WHEN severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical,
            SUM(CASE WHEN severity = 'MAJOR' THEN 1 ELSE 0 END) AS major,
            SUM(CASE WHEN severity = 'MINOR' THEN 1 ELSE 0 END) AS minor
        FROM neo4j_maintenance_events
        GROUP BY aircraft_id
    )
    SELECT
        a.tail_number,
        a.model,
        a.operator,
        COALESCE(m.total_events, 0) AS maint_events,
        COALESCE(m.critical, 0) AS critical,
        COALESCE(m.major, 0) AS major,
        COALESCE(m.minor, 0) AS minor,
        s.avg_egt AS avg_egt_c,
        s.max_egt AS max_egt_c,
        s.avg_vibration AS avg_vib_ips,
        s.max_vibration AS max_vib_ips
    FROM aircraft_ref a
    LEFT JOIN maintenance_summary m ON a.aircraft_id = m.aircraft_id
    LEFT JOIN sensor_health s ON a.aircraft_id = s.aircraft_id
    ORDER BY m.total_events DESC NULLS LAST
""")

result.show(20, truncate=False)
assert result.count() > 0, "Expected aircraft rows"
print("[PASS] Sensor + maintenance correlation works via views")

In [ ]:
# =============================================================================
# FEDERATED QUERY 3: Flight Operations + Engine Performance
# =============================================================================
# Correlates flight activity (Neo4j) with engine sensor data (Delta).
# Equivalent to Section 4 of federated_lakehouse_query.ipynb.

print("Federated Query 3: Flight Operations + Engine Performance")
print("Delta: sensor_readings (Engine sensors), sensors, systems, aircraft")
print("Neo4j: neo4j_flights (view)")
print("=" * 90)

result = spark.sql("""
    WITH aircraft_ref AS (
        SELECT `:ID(Aircraft)` AS aircraft_id, tail_number, model, operator
        FROM aircraft
    ),
    flight_activity AS (
        SELECT
            aircraft_id,
            COUNT(*) AS total_flights,
            COUNT(DISTINCT origin) AS unique_origins,
            COUNT(DISTINCT destination) AS unique_destinations
        FROM neo4j_flights
        GROUP BY aircraft_id
    ),
    engine_health AS (
        SELECT
            sys.aircraft_id,
            ROUND(AVG(CASE WHEN sen.type = 'EGT' THEN r.value END), 1) AS avg_egt,
            ROUND(AVG(CASE WHEN sen.type = 'FuelFlow' THEN r.value END), 2) AS avg_fuel_flow,
            ROUND(AVG(CASE WHEN sen.type = 'N1Speed' THEN r.value END), 0) AS avg_n1_speed
        FROM sensor_readings r
        JOIN sensors sen ON r.sensor_id = sen.`:ID(Sensor)`
        JOIN systems sys ON sen.system_id = sys.`:ID(System)`
        WHERE sys.type = 'Engine'
        GROUP BY sys.aircraft_id
    )
    SELECT
        a.tail_number,
        a.model,
        a.operator,
        f.total_flights,
        f.unique_origins AS origins,
        f.unique_destinations AS destinations,
        e.avg_egt AS avg_egt_c,
        e.avg_fuel_flow AS fuel_kgs,
        e.avg_n1_speed AS n1_rpm
    FROM aircraft_ref a
    JOIN flight_activity f ON a.aircraft_id = f.aircraft_id
    JOIN engine_health e ON a.aircraft_id = e.aircraft_id
    ORDER BY f.total_flights DESC
""")

result.show(20, truncate=False)
assert result.count() > 0, "Expected aircraft with flights and engine data"
print("[PASS] Flight ops + engine performance works via views")

In [ ]:
# =============================================================================
# FEDERATED QUERY 4: Fleet Health Dashboard
# =============================================================================
# Comprehensive view combining ALL data sources — Delta tables + Neo4j views.
# Equivalent to Section 5 of federated_lakehouse_query.ipynb but using only
# UC federation (no Spark Connector).

print("Federated Query 4: Fleet Health Dashboard")
print("Delta: sensor_readings, sensors, systems, aircraft")
print("Neo4j: neo4j_maintenance_events + neo4j_flights (views)")
print("=" * 100)

result = spark.sql("""
    WITH aircraft_ref AS (
        SELECT `:ID(Aircraft)` AS aircraft_id, tail_number, model, manufacturer, operator
        FROM aircraft
    ),
    sensor_stats AS (
        SELECT
            sys.aircraft_id,
            ROUND(AVG(CASE WHEN sen.type = 'EGT' THEN r.value END), 1) AS avg_egt,
            ROUND(AVG(CASE WHEN sen.type = 'Vibration' THEN r.value END), 4) AS avg_vib,
            ROUND(AVG(CASE WHEN sen.type = 'FuelFlow' THEN r.value END), 2) AS avg_fuel,
            COUNT(*) AS reading_count
        FROM sensor_readings r
        JOIN sensors sen ON r.sensor_id = sen.`:ID(Sensor)`
        JOIN systems sys ON sen.system_id = sys.`:ID(System)`
        GROUP BY sys.aircraft_id
    ),
    maint AS (
        SELECT
            aircraft_id,
            COUNT(*) AS events,
            SUM(CASE WHEN severity = 'CRITICAL' THEN 1 ELSE 0 END) AS critical
        FROM neo4j_maintenance_events
        GROUP BY aircraft_id
    ),
    flights AS (
        SELECT aircraft_id, COUNT(*) AS flight_count
        FROM neo4j_flights
        GROUP BY aircraft_id
    )
    SELECT
        a.tail_number,
        a.model,
        a.operator,
        COALESCE(f.flight_count, 0) AS flights,
        COALESCE(m.events, 0) AS maint_events,
        COALESCE(m.critical, 0) AS critical,
        s.avg_egt AS egt_c,
        s.avg_vib AS vib_ips,
        s.avg_fuel AS fuel_kgs,
        s.reading_count AS readings
    FROM aircraft_ref a
    LEFT JOIN flights f ON a.aircraft_id = f.aircraft_id
    LEFT JOIN maint m ON a.aircraft_id = m.aircraft_id
    LEFT JOIN sensor_stats s ON a.aircraft_id = s.aircraft_id
    ORDER BY COALESCE(m.critical, 0) DESC, COALESCE(m.events, 0) DESC
""")

result.show(20, truncate=False)
assert result.count() > 0, "Expected fleet health rows"
print("[PASS] Fleet health dashboard works entirely through UC federation")

In [ ]:
# =============================================================================
# FEDERATED QUERY 5: High EGT Aircraft with Critical Maintenance
# =============================================================================
# The kind of question a Genie agent would answer:
# "Which aircraft with high EGT readings also had critical maintenance events?"

print("Federated Query 5: High EGT + Critical Maintenance")
print("Natural language equivalent: 'Which aircraft with high EGT also had critical maintenance?'")
print("=" * 100)

result = spark.sql("""
    WITH aircraft_egt AS (
        SELECT
            sys.aircraft_id,
            ROUND(AVG(r.value), 1) AS avg_egt,
            ROUND(MAX(r.value), 1) AS max_egt
        FROM sensor_readings r
        JOIN sensors sen ON r.sensor_id = sen.`:ID(Sensor)`
        JOIN systems sys ON sen.system_id = sys.`:ID(System)`
        WHERE sen.type = 'EGT'
        GROUP BY sys.aircraft_id
    ),
    critical_maint AS (
        SELECT
            aircraft_id,
            COUNT(*) AS critical_count,
            COLLECT_LIST(fault) AS faults
        FROM neo4j_maintenance_events
        WHERE severity = 'CRITICAL'
        GROUP BY aircraft_id
    )
    SELECT
        a.tail_number,
        a.model,
        a.operator,
        e.avg_egt AS avg_egt_c,
        e.max_egt AS max_egt_c,
        cm.critical_count,
        cm.faults
    FROM aircraft a
    JOIN aircraft_egt e ON a.`:ID(Aircraft)` = e.aircraft_id
    JOIN critical_maint cm ON a.`:ID(Aircraft)` = cm.aircraft_id
    ORDER BY e.avg_egt DESC
""")

result.show(20, truncate=False)
print(f"Found {result.count()} aircraft with both EGT data and critical maintenance")
print("[PASS] Cross-source correlation query works via views")

In [ ]:
# =============================================================================
# FEDERATED QUERY 6: Route Coverage + Engine Health
# =============================================================================
# Uses the graph traversal view (Flight→Airport) joined with Delta sensor data.
# "Which departure airports see aircraft with the highest average EGT?"

print("Federated Query 6: Route Coverage + Engine Health")
print("Neo4j: neo4j_flight_airports (graph traversal view)")
print("Delta: sensor_readings, sensors, systems")
print("=" * 90)

result = spark.sql("""
    WITH airport_aircraft AS (
        SELECT DISTINCT airport_code, airport_name, aircraft_id
        FROM neo4j_flight_airports
    ),
    aircraft_egt AS (
        SELECT
            sys.aircraft_id,
            ROUND(AVG(r.value), 1) AS avg_egt
        FROM sensor_readings r
        JOIN sensors sen ON r.sensor_id = sen.`:ID(Sensor)`
        JOIN systems sys ON sen.system_id = sys.`:ID(System)`
        WHERE sen.type = 'EGT'
        GROUP BY sys.aircraft_id
    )
    SELECT
        aa.airport_code,
        aa.airport_name,
        COUNT(DISTINCT aa.aircraft_id) AS aircraft_count,
        ROUND(AVG(e.avg_egt), 1) AS avg_fleet_egt_c
    FROM airport_aircraft aa
    JOIN aircraft_egt e ON aa.aircraft_id = e.aircraft_id
    GROUP BY aa.airport_code, aa.airport_name
    ORDER BY avg_fleet_egt_c DESC
""")

result.show(truncate=False)
assert result.count() > 0, "Expected airport-level engine health data"
print("[PASS] Graph traversal view + Delta sensor join works")

---

## Summary

This notebook materializes Neo4j graph data as **managed Delta tables** in Unity Catalog,
making it queryable alongside lakehouse tables with standard SQL — GROUP BY, ORDER BY,
JOINs, and aggregations all work seamlessly.

### Neo4j Tables Materialized

| Table | Neo4j Source | Description |
|-------|-------------|-------------|
| `neo4j_maintenance_events` | `MaintenanceEvent` nodes | Maintenance events with severity, fault, corrective action |
| `neo4j_flights` | `Flight` nodes | Flight operations with origin/destination |
| `neo4j_airports` | `Airport` nodes | Airport reference data (iata, name, city, country) |
| `neo4j_flight_airports` | Flights + Airports | Flight-to-departure-airport mapping |

All tables are in: `aws-databricks-neo4j-lab.lakehouse`

### SQL Tests Passed

| Test | Operation | Status |
|------|-----------|--------|
| 1 | GROUP BY on Neo4j table | PASS |
| 2 | ORDER BY on Neo4j table | PASS |
| 3 | WHERE filter on Neo4j table | PASS |
| 4 | Aggregations (COUNT, SUM, CASE) | PASS |
| 5 | DISTINCT on flight-airport table | PASS |
| 6 | GROUP BY on flight-airport table | PASS |

### Federated Queries Passed

| Query | Neo4j Tables | Delta Tables | Equivalent |
|-------|-------------|-------------|------------|
| Fleet Summary | all tables | sensor_readings, sensors | Section 2 of federated_lakehouse_query |
| Sensor + Maintenance | maintenance_events | all 4 Delta tables | Section 3 |
| Flight Ops + Engine | flights | sensor_readings, sensors, systems | Section 4 |
| Fleet Health Dashboard | maintenance + flights | all 4 Delta tables | Section 5 |
| High EGT + Critical Maint | maintenance_events | sensor_readings, sensors, systems | New |
| Route + Engine Health | flight_airports | sensor_readings, sensors, systems | New |

### Agent-Ready

These materialized tables make the data **Genie-ready**. A Genie space configured with
all 8 tables can answer natural language questions that federate across Neo4j and Delta:

```
NL → SQL (Genie) → Spark SQL → Delta tables (lakehouse + materialized Neo4j data)
```

No Spark Connector. No Python drivers. Re-run this notebook to refresh Neo4j data.

### Next Steps

- Set up a **Genie space** with all 8 tables and example SQL instructions
- Connect Genie to an **agent** via MCP server or Conversation API
- See [FEDERATED_AGENTS.md](../FEDERATED_AGENTS.md) for the full agent architecture

### References

- [GUIDE_NEO4J_UC.md](../GUIDE_NEO4J_UC.md) — Full UC JDBC integration guide
- [FEDERATED_AGENTS.md](../FEDERATED_AGENTS.md) — Agent architecture with federation
- [federated_lakehouse_query.ipynb](federated_lakehouse_query.ipynb) — Original federated queries (Spark Connector + remote_query)
- [Neo4j JDBC SQL2Cypher](https://neo4j.com/docs/jdbc-manual/current/sql2cypher/) — SQL translation rules
- [Databricks remote_query()](https://docs.databricks.com/sql/language-manual/functions/remote_query) — Table-valued function reference